## Analysis of geographical information on the Repertoire

The Repertoir includes two lists of Jesuits residences in China.

One reports to 1644 [Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644)](../sources/dehergne-locations-1644.cli)
and the second to 1701  [VII. Carte des résidences de Chine en l'année 1701](../sources/dehergne-locations-1701.cli).


In [40]:

from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(
    kleio_version='12.6.576')
tlnb.print_info(show_token=True)

Timelink version: 1.1.14
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: POhms0fr7ak18OyfZj09jqx26mhD1CGk
Kleio server URL: http://127.0.0.1:8094
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: strange_elgamal
Kleio version requested: 12.6.576
Kleio server version: 12.6.576 (2024-09-09 17:57:04)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=12.6.576, postgres_image=postgres, postgres_version=latest)


In [41]:
from timelink.kleio.utilities import get_extra_info

def get_wikidata_id(obs, if_missing=''):
    """ Check the obs field for wikidata links

    Returns a tuple of the cleaned comment and the wikidata id"""
    clean_obs, extra_info = get_extra_info(obs)
    name_comment = extra_info.get('name', {}).get('comment','')
    name_original = extra_info.get('name', {}).get('original','')

    pattern = r'@wikidata\:\s*(Q[0-9]*)'
    wikidata_in_comment = re.findall(pattern, name_comment)
    comment_without_wikidata = re.sub(pattern, '', name_comment)
    # Sometimes the wikidata id is in the original name
    wikidata_in_original = re.findall(pattern, name_original)
    original_without_wikidata = re.sub(pattern, '', name_original)
    return clean_obs + comment_without_wikidata + original_without_wikidata, wikidata_in_comment[0] if wikidata_in_comment else wikidata_in_original[0] if wikidata_in_original else if_missing



## List of residences in 1644

In [42]:
import re
import pandas as pd
from sqlalchemy import select

geo1, geo2, geo3 = tlnb.db.get_model(['geo1','geo2','geo3'])
stmt = select(geo1).where(geo1.inside == 'deh-chre-1644')

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province.obs, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            comment, wikidata = get_wikidata_id(fou.obs, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien.obs, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})


Chekiang Q16967 Tche-kiang, hoje:Zhejiang, 浙江,  @dehergne:396
  Hangchou Q4970 Hang-tcheou, hoje: Hangzhou, 杭州, 
    Fuyang Q1011103 Fou-yang, hoje:Fuyang, 富阳, 
    Jenho No wikidata no wikidata.Jen-houo, hoje: Renhe, 仁和县, Historical county name. coordinate: 30.448897N, 120.307504E
  Chüchow Q58235 in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州".K'iu-tcheou, hoje:Quzhou, 衢州, , in the Chinese translation it is recognized as “遂州”, which is wrong, both phonetically and geographically. In Dehergne(1957), it is noted as "衢州".
  Huchow Q42664 Hou-tcheou, hoje: Huzhou, 湖州, 
    Tehtsing Q1191987 "Tehtsing du Huchow, Té-ts'ing;Teching, hoje: Deqing, 德清, "
  Kashing Q58178 Kia-hing, hoje:Jiaxing, 嘉兴, 
    Kashan Q1361347 """
               The place name in the book Dehergne(1973) is "Kaoshan", and in the Chinese translation it is recognized as “高山”. But this place cannot be found. According to Deh

In [43]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1644_df = pd.DataFrame(place_list)
places_1644_df

,province,fou,name,wikidata,comment
0,Chekiang,,Chekiang,Q16967,"Tche-kiang, hoje:Zhejiang, 浙江, @dehergne:396"
1,Chekiang,Hangchou,Hangchou,Q4970,"Hang-tcheou, hoje: Hangzhou, 杭州,"
2,Chekiang,Hangchou,Fuyang,Q1011103,"Fou-yang, hoje:Fuyang, 富阳,"
3,Chekiang,Hangchou,Jenho,No wikidata,"no wikidata.Jen-houo, hoje: Renhe, 仁和县, Histor..."
4,Chekiang,Chüchow,Chüchow,Q58235,in the Chinese translation it is recognized as...
5,Chekiang,Huchow,Huchow,Q42664,"Hou-tcheou, hoje: Huzhou, 湖州,"
6,Chekiang,Huchow,Tehtsing,Q1191987,"""Tehtsing du Huchow, Té-ts'ing;Teching, hoje: ..."
7,Chekiang,Kashing,Kashing,Q58178,"Kia-hing, hoje:Jiaxing, 嘉兴,"
8,Chekiang,Kashing,Kashan,Q1361347,"""""""\n The place name in the book..."
9,Chekiang,Kashing,Tangsi,Q10931032,"""T'ang-k'i Tangchi"", hoje:Tangqi, 塘栖 , in the ..."


In [44]:
places_1644_df.to_excel('../inferences/places-1644.xlsx', index=False)

## 1701

In [45]:
stmt = select(geo1).where(geo1.inside == 'deh-chre-1701')

place_list = []

with tlnb.db.session() as session:
    result = session.execute(stmt).fetchall()
    for province, in result:
        comment, wikidata = get_wikidata_id(province.obs, if_missing='No wikidata')
        print(province.name, wikidata, comment)
        place_list.append({'province': province.name,'fou':'',  'name': province.name, 'wikidata': wikidata, 'comment': comment})
        fous = session.execute(select(geo2).where(geo2.inside == province.id)).fetchall()
        for fou, in fous:
            comment, wikidata = get_wikidata_id(fou.obs, if_missing='No wikidata')
            print(' ', fou.name,  wikidata, comment)
            place_list.append({'province': province.name, 'fou':fou.name,  'name': fou.name, 'wikidata': wikidata, 'comment': comment})
            geo3s = session.execute(select(geo3).where(geo3.inside == fou.id)).fetchall()
            for tcheou_hien, in geo3s:
                comment, wikidata = get_wikidata_id(tcheou_hien.obs, if_missing='No wikidata')
                print('   ', tcheou_hien.name, wikidata,comment,  )
                place_list.append({'province': province.name, 'fou':fou.name,  'name': tcheou_hien.name, 'wikidata': wikidata, 'comment': comment})


Chekiang Q16967 Zhejiang， CHEKIANG (actuel ZHEJIANG)
  Hangchou Q4970 Hangzhou, 
    Haining No wikidata 
  Kashing Q58178 Jiaxing, 
  Kinhwa Q58210 Jinhua, 
    Lanchi Q1023793 Lanxi, 
  Ningpo Q42780 Ningbo, 
    Yenchow No wikidata 
Fukien Q41705 Fujian, 
  Foochow Q68481 Fuzhou, 
    Lienkong Q204827 Lianjiang, 
  Changchow Q68814 Zhangzhou, 
  Chüanchow Q68695 Quanzhou, 
    Amoy Q68744 Xiamen, 
  Funing Q241877 (alors tcheou indépendant)Funing, 
    Fuan Q1374581 Fu'an, 
  Hinghwa Q17498990 Xinghua, 
  Kienning Q11065314 Kienning fou, Jianning, 
    Pucheng No wikidata 
  Shaowu Q1025451 Shaowu, 
    Kienning hien Q781559 Jianning Xian, 
  Tingchow Q7808213 Tingzhou, 
    Shanghang No wikidata 
  Yenping Q1337959 Yanping, 
    Tsianglo No wikidata 
Honan Q43684 Henan, 
  Kaifeng Q208471 Kaifeng, 
  Kweiteh Q404817 Guide, 
Hukwang Q2235155 (actuel HUNAN + HUBEI)Huguang, 
  Wuchang Q1014420 Wuchang, 
  Anlu Q557934 Anlu, 
  Changsha No wikidata 
    Siangtan No wikidata 
  Chengcho

In [46]:
pd.set_option('display.max_rows', 300)
# create a dataframe from the list
places_1701_df = pd.DataFrame(place_list)
places_1701_df

,province,fou,name,wikidata,comment
0,Chekiang,,Chekiang,Q16967,Zhejiang， CHEKIANG (actuel ZHEJIANG)
1,Chekiang,Hangchou,Hangchou,Q4970,"Hangzhou,"
2,Chekiang,Hangchou,Haining,No wikidata,
3,Chekiang,Kashing,Kashing,Q58178,"Jiaxing,"
4,Chekiang,Kinhwa,Kinhwa,Q58210,"Jinhua,"
5,Chekiang,Kinhwa,Lanchi,Q1023793,"Lanxi,"
6,Chekiang,Ningpo,Ningpo,Q42780,"Ningbo,"
7,Chekiang,Ningpo,Yenchow,No wikidata,
8,Fukien,,Fukien,Q41705,"Fujian,"
9,Fukien,Foochow,Foochow,Q68481,"Fuzhou,"


In [47]:
places_1701_df.to_excel('../inferences/places-1701.xlsx', index=False)